# Model Evaluation - BERT 

In this notebook, i'll combine the results of the 5 different BERT models. Initially I did some averaging but the results were not ideal and hence I manually combined them to observe the upper limit of this BERT implementation.

We read the predictions, labels into dataframes.

In [62]:
import pandas as pd 

y_all_0_df = pd.read_csv('y_all_0.csv',sep=',',header=None)
y_all_500_df = pd.read_csv('y_all_500.csv',sep=',',header=None)
y_all_1000_df = pd.read_csv('y_all_1000.csv',sep=',',header=None)
y_all_1500_df = pd.read_csv('y_all_1500.csv',sep=',',header=None)
y_all_2000_df = pd.read_csv('y_all_2000.csv',sep=',',header=None)

y_hat_0_df = pd.read_csv('y_hat_0.csv',sep=',',header=None)
y_hat_500_df = pd.read_csv('y_hat_500.csv',sep=',',header=None)
y_hat_1000_df = pd.read_csv('y_hat_1000.csv',sep=',',header=None)
y_hat_1500_df = pd.read_csv('y_hat_1500.csv',sep=',',header=None)
y_hat_2000_df = pd.read_csv('y_hat_2000.csv',sep=',',header=None)

yhat_raws_0_df = pd.read_csv('yhat_raws_0.csv',sep=',',header=None)
yhat_raws_500_df = pd.read_csv('yhat_raws_500.csv',sep=',',header=None)
yhat_raws_1000_df = pd.read_csv('yhat_raws_1000.csv',sep=',',header=None)
yhat_raws_1500_df = pd.read_csv('yhat_raws_1500.csv',sep=',',header=None)
yhat_raws_2000_df = pd.read_csv('yhat_raws_2000.csv',sep=',',header=None)


There are 11,404 labels in y_all.

In [63]:
y_all_0_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11404 entries, 0 to 11403
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       11404 non-null  float64
 1   1       11404 non-null  float64
 2   2       11404 non-null  float64
 3   3       11404 non-null  float64
 4   4       11404 non-null  float64
 5   5       11404 non-null  float64
 6   6       11404 non-null  float64
 7   7       11404 non-null  float64
 8   8       11404 non-null  float64
 9   9       11404 non-null  float64
 10  10      11404 non-null  float64
 11  11      11404 non-null  float64
 12  12      11404 non-null  float64
 13  13      11404 non-null  float64
 14  14      11404 non-null  float64
 15  15      11404 non-null  float64
 16  16      11404 non-null  float64
 17  17      11404 non-null  float64
 18  18      11404 non-null  float64
 19  19      11404 non-null  float64
 20  20      11404 non-null  float64
 21  21      11404 non-null  float64
 22

There aere 11,404 labels in y_all_500 as well. This should be identical.

In [64]:
y_all_500_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11404 entries, 0 to 11403
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       11404 non-null  float64
 1   1       11404 non-null  float64
 2   2       11404 non-null  float64
 3   3       11404 non-null  float64
 4   4       11404 non-null  float64
 5   5       11404 non-null  float64
 6   6       11404 non-null  float64
 7   7       11404 non-null  float64
 8   8       11404 non-null  float64
 9   9       11404 non-null  float64
 10  10      11404 non-null  float64
 11  11      11404 non-null  float64
 12  12      11404 non-null  float64
 13  13      11404 non-null  float64
 14  14      11404 non-null  float64
 15  15      11404 non-null  float64
 16  16      11404 non-null  float64
 17  17      11404 non-null  float64
 18  18      11404 non-null  float64
 19  19      11404 non-null  float64
 20  20      11404 non-null  float64
 21  21      11404 non-null  float64
 22

We do a simple averaging of the 5 different BERT predictions and observe that the results is not that ideal. Instead, will explore the upper performance limits of this approach.

In [187]:
k = 8
# num true labels in top k predictions / k
sortd = np.argsort(yhat_raws)[:, ::-1]
#print(sortd[0])
topk = sortd[:, :k]
#print(topk[0])
selected = True
# get precision at k for each example
vals = []

for i, y_all_tk in enumerate(y_all):
    result = 0.2*yhat_raws_0_df.to_numpy() + 0.2*yhat_raws_500_df.to_numpy() +0.2*yhat_raws_1000_df.to_numpy() +0.2*yhat_raws_1500_df.to_numpy() +0.2*yhat_raws_2000_df.to_numpy()
    sortd = np.argsort(result)[:, ::-1]
    topk = sortd[:, :k]
    #print(topk[0])
    num_true_in_top_k = y_all_tk[topk[i]].sum()

    vals.append(num_true_in_top_k / k)
    
np.mean(vals)

0.35444078947368424

What the following code segment will do is to iterate through all the labels of the dataset. Select the BERT output (we have 5 to choose from) and choose the one with least errors. This is to obtain the upper performance limit of the model. 

In [163]:
y_optim = []
for index, row in enumerate(y_all):
    #print('new row')
    result1 = sum(row != round(yhat_raws_0_df.iloc[index,:]))
    result2 = sum(row != round(yhat_raws_500_df.iloc[index,:]))
    result3 = sum(row != round(yhat_raws_1000_df.iloc[index,:]))
    result4 = sum(row != round(yhat_raws_1500_df.iloc[index,:]))
    result5 = sum(row != round(yhat_raws_2000_df.iloc[index,:]))
    
    result = [result1, result2, result3, result4, result5]
    
    value = min(result)
    #print(result)
    #print(value)
    #print(result.index(value))
    
    if result.index(value) == 0:
        y_optim.append(yhat_raws_0_df.iloc[index,:])
    elif result.index(value) == 1:
        y_optim.append(yhat_raws_500_df.iloc[index,:])
    elif result.index(value) == 2:
        y_optim.append(yhat_raws_1000_df.iloc[index,:])
    elif result.index(value) == 3:
        y_optim.append(yhat_raws_1500_df.iloc[index,:])
    elif result.index(value) == 4:
        y_optim.append(yhat_raws_2000_df.iloc[index,:])
    #if index == 20:
        #break

The following code segment will select and optimize the 5 BERT raw outputs based on precision@k with a specified k value of 8.

In [172]:
k = 8
# num true labels in top k predictions / k
sortd = np.argsort(yhat_raws)[:, ::-1]
#print(sortd[0])
topk = sortd[:, :k]
#print(topk[0])
selected = True
# get precision at k for each example
vals = []
for i, y_all_tk in enumerate(y_all):
    if selected:
        sortd = np.argsort(yhat_raws_0_df.to_numpy())[:, ::-1]
        topk1 = sortd[:, :k]
        num_true_in_top_k1 = y_all_tk[topk1[i]].sum()

        sortd = np.argsort(yhat_raws_500_df.to_numpy())[:, ::-1]
        topk2 = sortd[:, :k]
        num_true_in_top_k2 = y_all_tk[topk2[i]].sum()

        sortd = np.argsort(yhat_raws_1000_df.to_numpy())[:, ::-1]
        topk3 = sortd[:, :k]
        num_true_in_top_k3 = y_all_tk[topk3[i]].sum()

        sortd = np.argsort(yhat_raws_1500_df.to_numpy())[:, ::-1]
        topk4 = sortd[:, :k]
        num_true_in_top_k4 = y_all_tk[topk4[i]].sum()

        sortd = np.argsort(yhat_raws_2000_df.to_numpy())[:, ::-1]
        topk5 = sortd[:, :k]
        num_true_in_top_k5 = y_all_tk[topk5[i]].sum()

        num_true_in_top_k = [num_true_in_top_k1, 
                             num_true_in_top_k2, 
                             num_true_in_top_k3, 
                             num_true_in_top_k4, 
                             num_true_in_top_k5]
        topk_len = [float(len(topk1[i])), 
                    float(len(topk2[i])), 
                    float(len(topk3[i])), 
                    float(len(topk4[i])), 
                    float(len(topk5[i]))]
        print(topk_len)
        if i == 10:
            break;
        #num_true_in_top_k.index(min(num_true_in_top_k))
        #print(num_true_in_top_k)
        #denom = len(tk)
        index = num_true_in_top_k.index(max(num_true_in_top_k))
    #print(num_true_in_top_k[index])
    #print(topk2[0])
    #print(np.unique(np.concatenate((topk1[0],topk2[0]))))
        vals.append(num_true_in_top_k[index] / topk_len[index])
    else:
        sortd = np.argsort(yhat_raws_0_df.to_numpy())[:, ::-1]
        topk1 = sortd[:, :k]

        sortd = np.argsort(yhat_raws_500_df.to_numpy())[:, ::-1]
        topk2 = sortd[:, :k]

        sortd = np.argsort(yhat_raws_1000_df.to_numpy())[:, ::-1]
        topk3 = sortd[:, :k]

        sortd = np.argsort(yhat_raws_1500_df.to_numpy())[:, ::-1]
        topk4 = sortd[:, :k]

        sortd = np.argsort(yhat_raws_2000_df.to_numpy())[:, ::-1]
        topk5 = sortd[:, :k]

        combined_topk = np.unique(np.concatenate((topk1[i],topk2[i],topk3[i],topk4[i],topk5[i])))
        
        num_true_in_top_k = y_all_tk[combined_topk].sum()
        vals.append(num_true_in_top_k / float(len(combined_topk)))
    
np.mean(vals)

[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]
[8.0, 8.0, 8.0, 8.0, 8.0]


0.4375

We'll need to convert these 3 variables (y_all, yhat_raws, y_hats) into numpy arrays for our subsequent codes to work.

In [81]:
y_all = y_all_0_df.to_numpy() 
yhat_raws = yhat_raws_0_df.to_numpy() #+ 0.3*yhat_raws_500_df.to_numpy() 
y_hats = yhat_raws.round()

In [164]:
y_hats = round(pd.DataFrame(y_optim)).to_numpy()
yhat_raws = pd.DataFrame(y_optim).to_numpy()

These are the utility functions used to calculate Macro/Micro AUC, Macro/Micro F1, Precision@k. We'll make use of them again.

In [14]:
import numpy as np
import pandas as pd
from collections import defaultdict
import csv
import os
import time
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc

# Helper class that I found online, it's pretty good. Just computes the average. 
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/
def union_size(yhat, y, axis):
    # axis=0 for label-level union (macro). axis=1 for instance-level
    return np.logical_or(yhat, y).sum(axis=axis).astype(float)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/
def intersect_size(yhat, y, axis):
    # axis=0 for label-level union (macro). axis=1 for instance-level
    return np.logical_and(yhat, y).sum(axis=axis).astype(float)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def macro_accuracy(yhat, y):
    num = intersect_size(yhat, y, 0) / (union_size(yhat, y, 0) + 1e-10)
    return np.mean(num)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def macro_precision(yhat, y):
    num = intersect_size(yhat, y, 0) / (yhat.sum(axis=0) + 1e-10)
    return np.mean(num)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def macro_recall(yhat, y):
    num = intersect_size(yhat, y, 0) / (y.sum(axis=0) + 1e-10)
    return np.mean(num)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def macro_f1(yhat, y):
    prec = macro_precision(yhat, y)
    rec = macro_recall(yhat, y)
    if prec + rec == 0:
        f1 = 0.
    else:
        f1 = 2 * (prec * rec) / (prec + rec)
    return f1

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def micro_accuracy(yhatmic, ymic):
    return intersect_size(yhatmic, ymic, 0) / union_size(yhatmic, ymic, 0)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def micro_precision(yhatmic, ymic):
    return intersect_size(yhatmic, ymic, 0) / yhatmic.sum(axis=0)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def micro_recall(yhatmic, ymic):
    return intersect_size(yhatmic, ymic, 0) / ymic.sum(axis=0)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def micro_f1(yhatmic, ymic):
    prec = micro_precision(yhatmic, ymic)
    rec = micro_recall(yhatmic, ymic)
    if prec + rec == 0:
        f1 = 0.
    else:
        f1 = 2 * (prec * rec) / (prec + rec)
    return f1

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def inst_precision(yhat, y):
    num = intersect_size(yhat, y, 1) / yhat.sum(axis=1)
    # correct for divide-by-zeros
    num[np.isnan(num)] = 0.
    return np.mean(num)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def inst_recall(yhat, y):
    num = intersect_size(yhat, y, 1) / y.sum(axis=1)
    # correct for divide-by-zeros
    num[np.isnan(num)] = 0.
    return np.mean(num)

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def inst_f1(yhat, y):
    prec = inst_precision(yhat, y)
    rec = inst_recall(yhat, y)
    f1 = 2 * (prec * rec) / (prec + rec)
    return f1

#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def precision_at_k(yhat_raw, y, k):
    # num true labels in top k predictions / k
    sortd = np.argsort(yhat_raw)[:, ::-1]
    topk = sortd[:, :k]

    # get precision at k for each example
    vals = []
    for i, tk in enumerate(topk):
        if len(tk) > 0:
            num_true_in_top_k = y[i, tk].sum()
            denom = len(tk)
            vals.append(num_true_in_top_k / float(denom))

    return np.mean(vals)


#  referenced from James Mullenbach, https://github.com/jamesmullenbach/caml-mimic/evaluation.py
def auc_metrics(yhat_raw, y, ymic):
    if yhat_raw.shape[0] <= 1:
        return
    fpr = {}
    tpr = {}
    roc_auc = {}
    # get AUC for each label individually
    relevant_labels = []
    auc_labels = {}
    for i in range(y.shape[1]):
        # only if there are true positives for this label
        if y[:, i].sum() > 0:
            fpr[i], tpr[i], _ = roc_curve(y[:, i], yhat_raw[:, i])
            if len(fpr[i]) > 1 and len(tpr[i]) > 1:
                auc_score = auc(fpr[i], tpr[i])
                if not np.isnan(auc_score):
                    auc_labels["auc_%d" % i] = auc_score
                    relevant_labels.append(i)

    # macro-AUC: just average the auc scores
    aucs = []
    for i in relevant_labels:
        aucs.append(auc_labels['auc_%d' % i])
    roc_auc['auc_macro'] = np.mean(aucs)

    # micro-AUC: just look at each individual prediction
    yhatmic = yhat_raw.ravel()
    fpr["micro"], tpr["micro"], _ = roc_curve(ymic, yhatmic)
    roc_auc["auc_micro"] = auc(fpr["micro"], tpr["micro"])

    return roc_auc

The following code segment will calculate the performance metrics.

In [165]:
#print('y_hats')
#print(y_hats.shape)
num_rows = int(y_hats.shape[0])
#num_cols = int(y_hats.shape[1])
#print(num_rows)
#print(num_cols)
num_rows_batch_1 = int(y_hats.shape[0]/5)
num_rows_batch_2 = int(y_hats.shape[0]/5*2)
num_rows_batch_3 = int(y_hats.shape[0]/5*3)
num_rows_batch_4 = int(y_hats.shape[0]/5*4)
num_rows_batch_5 = int(y_hats.shape[0]/5*5)

#y_hats = y_hats[0:num_rows_batch_1,:] #+ y_hats[num_rows_batch_1:num_rows_batch_2,:] + y_hats[num_rows_batch_2:num_rows_batch_3,:] + y_hats[num_rows_batch_3:num_rows_batch_4,:] + y_hats[num_rows_batch_4:num_rows,:]
#y_all = y_all[0:num_rows_batch_1,:] #+ y_all[num_rows_batch_1:num_rows_batch_2,:] + y_all[num_rows_batch_2:num_rows_batch_3,:] + y_all[num_rows_batch_3:num_rows_batch_4,:] + y_all[num_rows_batch_4:num_rows,:]
#yhat_raws = yhat_raws[0:num_rows_batch_1,:] #+ yhat_raws[num_rows_batch_1:num_rows_batch_2,:] + yhat_raws[num_rows_batch_2:num_rows_batch_3,:] + yhat_raws[num_rows_batch_3:num_rows_batch_4,:] + yhat_raws[num_rows_batch_4:num_rows,:]

#y_hats[y_hats > 1] = 1
#y_all[y_all > 1] = 1
#yhat_raws[yhat_raws > 1] = 1

mac_acc = macro_accuracy(y_hats, y_all)
mac_rec = macro_recall(y_hats, y_all)
mac_pre = macro_precision(y_hats, y_all)
mic_acc = micro_accuracy(y_hats.ravel(), y_all.ravel())
mic_rec = micro_recall(y_hats.ravel(), y_all.ravel())
mic_pre = micro_precision(y_hats.ravel(), y_all.ravel())
mic_f1 = micro_f1(y_hats.ravel(), y_all.ravel())
mac_f1 = macro_f1(y_hats, y_all)
auc_dict = auc_metrics(yhat_raws, y_all, y_all.ravel())
prec_at_5 = precision_at_k(yhat_raws, y_all, 5)
prec_at_8 = precision_at_k(yhat_raws, y_all, 8)
prec_at_15 = precision_at_k(yhat_raws, y_all, 15)

print('Macro Accuracy {mac_acc:.3f} \t'
      'Macro Recall {mac_rec:.3f} \t'
      'Macro Precision {mac_pre:.3f} \t'
      'Macro F1 {mac_f1:.3f} \t'
      'Macro AUC {mac_auc:.3f} \t'
      'Micro Accuracy {mic_acc:.3f} \t'
      'Micro Recall {mic_rec:.3f} \t'
      'Micro Precision {mic_pre:.3f} \t'
      'Micro F1 {mic_f1:.3f} \t'
      'Micro AUC {auc_micro:.3f} \t'
      'P@5 {prec_at_5:.3f} \t'
      'P@8 {prec_at_8:.3f} \t'
      'P@15 {prec_at_15:.3f} \t'.format(
    mac_acc=mac_acc,
    mac_pre=mac_pre,
    mac_rec=mac_rec,
    mac_f1=mac_f1,
    mac_auc=auc_dict['auc_macro'],
    mic_acc=mic_acc,
    mic_pre=mic_pre,
    mic_rec=mic_rec,
    mic_f1=mic_f1,
    auc_micro=auc_dict['auc_micro'],
    prec_at_5=prec_at_5,
    prec_at_8=prec_at_8,
    prec_at_15=prec_at_15))



Macro Accuracy 0.186 	Macro Recall 0.203 	Macro Precision 0.478 	Macro F1 0.285 	Macro AUC 0.801 	Micro Accuracy 0.259 	Micro Recall 0.273 	Micro Precision 0.834 	Micro F1 0.411 	Micro AUC 0.846 	P@5 0.480 	P@8 0.383 	P@15 0.269 	


# Closing remarks

Now that we observe that the upper limit of this current 5 BERT, 500 words each approach achieves a Macro AUC score of 0.801, Micro AUC score of 0.846, Macro F1 of 0.285, Micro F1 of 0.411, P@8 of 0.383, P@15 of 0.269. Probably will have to explore other implementations in order to achieve a better score compared to LR/CNN/BiGRU/CAML. 